## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

**Answer: `"version" : 
    "number" : "8.17.6",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"**



## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [1]:
import requests 

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

**Answer: `index`**

In [4]:
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# Check if the index exists and delete it if it does
if es_client.indices.exists(index=index_name):
    print(f"Index {index_name} exists, deleting.")
    es_client.indices.delete(index=index_name)

es_client.indices.create(index=index_name, body=index_settings)

Index course-questions exists, deleting.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
# Index documents with elasticsearch
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

**Answer: `44.50`**

In [9]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }


    response = es_client.search(index=index_name, body=search_query)
    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    # return result_docs

    return response

In [10]:
query = "How do execute a command on a Kubernetes pod?"

In [11]:
elastic_search(query=query)

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'Lquze5cB15xfWBTYDnhW', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'vauze5cB15xfWBTYD3i_', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kuberne

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

**Answer: `How do I copy files from a different folder into docker container’s working directory?`**

In [12]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }


    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [13]:
query = "How do copy a file to a Docker container?"

In [14]:
elastic_search(query=query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446

**Answer: `1446`**

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
query = "How do copy a file to a Docker container?"
search_results = elastic_search(query=query)
prompt = build_prompt(query, search_results)
len(prompt)

1446

In [17]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

**Answer: `320`**

In [18]:
# ! pip install tiktoken

In [19]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

# Encode the prompt into tokens
tokens = encoding.encode(prompt)

# Print token count
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 320


In [20]:
# backwards - just an example for token id 63842
encoding.decode_single_token_bytes(63842).decode("utf-8")

"You're"

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

**Answer: `'You can copy a file from your local machine into a running Docker container using the `docker cp` command. The basic syntax for this command is:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n``` \n\nThis will copy the specified file or directory to the specified path inside the container.'`**

In [21]:
from dotenv import load_dotenv

from litellm import completion
from typing import List, Dict

from openai import OpenAI

In [22]:
# Load environment variables and set API key
load_dotenv()

True

In [23]:
client = OpenAI()

In [24]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [25]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [26]:
query = "How do copy a file to a Docker container?"
rag(query)

'You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax for copying a file or directory from your local machine to a running Docker container is:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:
* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

**Answer:`$5.75`**

You can redo the calculations with the values you got in Q6 and Q7.

**Answer:`$3.24`**

In [27]:
# Pricing as of June 17, 2025
INPUT_COST_PER_1M = 5.00      # $5.00 per 1M input tokens
OUTPUT_COST_PER_1M = 20.00    # $20.00 per 1M output tokens

# Average token usage per request
input_tokens_per_request = 150
output_tokens_per_request = 250

# Number of requests
num_requests = 1000

# Calculate total tokens
total_input_tokens = input_tokens_per_request * num_requests
total_output_tokens = output_tokens_per_request * num_requests

# Calculate costs
input_cost = (total_input_tokens / 1_000_000) * INPUT_COST_PER_1M
output_cost = (total_output_tokens / 1_000_000) * OUTPUT_COST_PER_1M
total_cost = input_cost + output_cost

# Display result
print(f"Total input tokens: {total_input_tokens}")
print(f"Total output tokens: {total_output_tokens}")
print(f"Input cost: ${input_cost:.4f}")
print(f"Output cost: ${output_cost:.4f}")
print(f"Total cost for {num_requests} requests: ${total_cost:.4f}")


Total input tokens: 150000
Total output tokens: 250000
Input cost: $0.7500
Output cost: $5.0000
Total cost for 1000 requests: $5.7500


### Calculating for taken values from Q6 & Q7

In [28]:
query = "How do copy a file to a Docker container?"
output = rag(query)
len(output)

301

In [29]:
# Encode output into tokens
tokens = encoding.encode(output)

# Print token count
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 62


In [30]:
# Cost

query = "How do copy a file to a Docker container?"
output = rag(query)

input_tokens_per_request = 320
output_tokens_per_request = 82

# Number of requests
num_requests = 1000

# Calculate total tokens
total_input_tokens = input_tokens_per_request * num_requests
total_output_tokens = output_tokens_per_request * num_requests

# Calculate costs
input_cost = (total_input_tokens / 1_000_000) * INPUT_COST_PER_1M
output_cost = (total_output_tokens / 1_000_000) * OUTPUT_COST_PER_1M
total_cost = input_cost + output_cost

# Display result
print(f"Total input tokens: {total_input_tokens}")
print(f"Total output tokens: {total_output_tokens}")
print(f"Input cost: ${input_cost:.4f}")
print(f"Output cost: ${output_cost:.4f}")
print(f"Total cost for {num_requests} requests: ${total_cost:.4f}")

Total input tokens: 320000
Total output tokens: 82000
Input cost: $1.6000
Output cost: $1.6400
Total cost for 1000 requests: $3.2400
